## Introduction


An image classifier using the VGG16 pre-trained model, Also the evaluate and predictions was done alongside ResNet50

## Download Data

In [1]:
## get the data
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2021-02-10 22:48:09--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  13.0MB/s    in 23s     

2021-02-10 22:48:32 (10.9 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [ ]:
!unzip -o concrete_data_week4.zip

Archive:  concrete_data_week4.zip
   creating: concrete_data_week4/
   creating: concrete_data_week4/valid/
   creating: concrete_data_week4/valid/positive/
  inflating: concrete_data_week4/valid/positive/16679_1.jpg  
   creating: __MACOSX/concrete_data_week4/
   creating: __MACOSX/concrete_data_week4/valid/
   creating: __MACOSX/concrete_data_week4/valid/positive/
  inflating: __MACOSX/concrete_data_week4/valid/positive/._16679_1.jpg  
  inflating: concrete_data_week4/valid/positive/19463.jpg  
  inflating: __MACOSX/concrete_data_week4/valid/positive/._19463.jpg  
  inflating: concrete_data_week4/valid/positive/19041_1.jpg  
  inflating: __MACOSX/concrete_data_week4/valid/positive/._19041_1.jpg  
  inflating: concrete_data_week4/valid/positive/19004_1.jpg  
  inflating: __MACOSX/concrete_data_week4/valid/positive/._19004_1.jpg  
  inflating: concrete_data_week4/valid/positive/18041_1.jpg  
  inflating: __MACOSX/concrete_data_week4/valid/positive/._18041_1.jpg  
  inflating: concrete_

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

A classifier using the VGG16 pre-trained model. Just like the ResNet50 model

## Import Libraries and Packages

In [3]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import ResNet50
from keras.applications import VGG16
from keras.applications.resnet50 import preprocess_input

## Define Global Constants


In [4]:
num_classes = 2

#VGG16 was originally trained on 224 × 224 images, so the images were resized.
image_resize = 224

batch_size_training = 10
batch_size_validation = 10

## ImageDataGenerator Instances

In [3]:
#ImageDataGenerator for the training set and another one for the validation set. 

vgg_data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

vgg_train_generator = vgg_data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

vgg_validation_generator = vgg_data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')


Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


## Building, Compiling and Fitting Model


In [4]:
#A sequential model using Keras and adding VGG16 model to it and dense layer.
vgg_model = Sequential()

vgg_model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

vgg_model.add(Dense(num_classes, activation='softmax'))

vgg_model.layers[0].trainable = False

vgg_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [5]:
#Compiling the mode using the adam optimizer and the categorical_crossentropy loss function.
vgg_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
steps_per_epoch_training = len(vgg_train_generator)
steps_per_epoch_validation = len(vgg_validation_generator)
num_epochs = 2

In [7]:
#Fitting the model on the augmented data using the ImageDataGenerators.

vgg_fit_history = vgg_model.fit_generator(
    vgg_train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=vgg_validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
3001/3001 [==============================] - 11713s 4s/step - loss: 0.0827 - accuracy: 0.9696 - val_loss: 0.0081 - val_accuracy: 0.9976
Epoch 2/2
3001/3001 [==============================] - 31213s 10s/step - loss: 0.0072 - accuracy: 0.9984 - val_loss: 0.0058 - val_accuracy: 0.9981


In [10]:
#Saving the model
vgg_model.save("crachh_vgg_model.h5")

<a id="item43"></a>

## Evaluation

In [10]:
#Loading the saved model that was built using the ResNet50 model and VGG16. 
from keras.models import load_model
resnet_model = load_model("classifier_resnet_model.h5")
vgg_model = load_model("crachh_vgg_model.h5")

In [11]:
#ImageDataGenerator for the test set.
da_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

test_generator = da_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical',
    shuffle = False)


Found 500 images belonging to 2 classes.


In [12]:
# VGG16 performance
vgg_peformance = vgg_model.evaluate(test_generator)
print("Performance of the VGG16-trained model")
print("Loss: {}".format(round(vgg_peformance[0], 5)))
print("Accuracy: {}".format(round(vgg_peformance[1], 5)))

50/50 [==============================] - 114s 2s/step - loss: 0.0033 - accuracy: 1.0000
Performance of the VGG16-trained model
Loss: 0.0033
Accuracy: 1.0


In [15]:
ResNet50 Performance
res_peformance = resnet_model.evaluate(test_generator)
print("Performance of the ResNet-trained model")
print("Loss: {}".format(round(res_peformance[0], 5)))
print("Accuracy: {}".format(round(res_peformance[1], 5)))

50/50 [==============================] - 46s 891ms/step - loss: 0.0016 - accuracy: 1.0000
Performance of the ResNet-trained model
Loss: 0.00156
Accuracy: 1.0


## Predictions

In [21]:
import numpy as np 
def predictions(args):
    for i in args:
        j = np.argmax(i)
        if (j == 0):
            print("Negative")
        else:
            print("Positive")

In [22]:
#Prediction for the first images in testset  using VGG16
predictions(vgg_model.predict_generator(test_generator)[0:5])

Negative
Negative
Negative
Negative
Negative


In [23]:
#Prediction for the first images in testset  using Resnet50
predictions(resnet_model.predict_generator(test_generator)[0:5])

Negative
Negative
Negative
Negative
Negative
